In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# Pull in Data

In [2]:
%%time
data = pd.read_csv("C:\\Users\\caleb\\PycharmProjects\\Sec_Project\\Data\\Kaggle-data.csv")

Wall time: 1.22 s


C:\Users\caleb\PycharmProjects\ditto\ven\lib\site-packages\IPython\core\magics\execution.py:1335: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


In [3]:
data.shape

(216352, 58)

In [4]:
data.head(4)

,ID,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate,Unnamed: 57
0,1,b69acb3bb133974e48229627663f96d4,332,224,8450,8.0,0,16896,8192,0,...,3.492126,3.492126,3.492126,864.0,864.0,864,72,0,1,NaN
1,2,1cbee4b3725629bd0aa6ac2ff500925f,332,224,258,9.0,0,84480,25600,0,...,3.486827,3.486827,3.486827,892.0,892.0,892,72,0,1,NaN
2,3,b7027cf0cd31c820928950cbfe7e91ef,332,224,8450,8.0,0,4608,3584,0,...,3.517270,3.517270,3.517270,952.0,952.0,952,72,0,1,NaN
3,4,156a0bb069f94d1e7c2508318805f2a4,332,224,8450,10.0,0,108544,15872,0,...,3.270559,3.034188,3.506931,1032.0,972.0,1092,72,0,1,NaN


In [5]:
data.columns

Index(['ID', 'md5', 'Machine', 'SizeOfOptionalHeader', 'Characteristics',
       'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
       'SizeOfInitializedData', 'SizeOfUninitializedData',
       'AddressOfEntryPoint', 'BaseOfCode', 'BaseOfData', 'ImageBase',
       'SectionAlignment', 'FileAlignment', 'MajorOperatingSystemVersion',
       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'SizeOfImage',
       'SizeOfHeaders', 'CheckSum', 'Subsystem', 'DllCharacteristics',
       'SizeOfStackReserve', 'SizeOfStackCommit', 'SizeOfHeapReserve',
       'SizeOfHeapCommit', 'LoaderFlags', 'NumberOfRvaAndSizes', 'SectionsNb',
       'SectionsMeanEntropy', 'SectionsMinEntropy', 'SectionsMaxEntropy',
       'SectionsMeanRawsize', 'SectionsMinRawsize', 'SectionMaxRawsize',
       'SectionsMeanVirtualsize', 'SectionsMinVirtualsize',
       'SectionMaxVirtualsize', 'ImportsNbDLL', 'ImportsNb',
       'Imports

# Clean Data

In [6]:
columns_to_drop = [
   "md5"
    ,"Machine"
    ,"Unnamed: 57"
]

In [7]:
clean_data = data.drop(columns=columns_to_drop)

In [8]:
for col in clean_data.columns:
    if clean_data[col].isnull().sum() != 0 or not str(clean_data[col].dtypes) in ["int64","float64"]:
        print(f"Dimension: {col}")
        print(f"Nulls: {clean_data[col].isnull().sum()}")
        print(f"Type: {clean_data[col].dtypes}")
        print(f"Unique Values: {clean_data[col].nunique()}")

Dimension: MajorLinkerVersion
Nulls: 1
Type: float64
Unique Values: 67


Because this is just a single unit will drop

In [9]:
clean_data[clean_data["MajorLinkerVersion"].isnull()]["legitimate"]

13    1
Name: legitimate, dtype: int64

In [10]:
clean_data = clean_data.dropna()

In [11]:
clean_data.shape

(216351, 55)

# Prepare for Modeling 

In [12]:
_id = np.array(clean_data['ID'])

In [13]:
labels = np.array(clean_data['legitimate'])

In [14]:
features= clean_data.drop(columns=['ID','legitimate'], axis = 1)

In [15]:
feature_list = list(features.columns)

In [16]:
features = np.array(features)

## Train/Test Split

In [17]:
train_features, test_features, train_labels, test_labels = \
train_test_split(features, labels, test_size = 0.25, random_state = 1202)

# Model 

## Create Model

In [18]:
%%time
rf = RandomForestClassifier(n_estimators = 100, random_state = 1202)

Wall time: 0 ns


In [19]:
%%time
rf.fit(train_features, train_labels)

Wall time: 47.1 s


RandomForestClassifier(random_state=1202)

## Test Model

In [20]:
predictions = rf.predict(test_features)

In [26]:
len(predictions) == len(test_labels)

True

In [25]:
len(predictions)

54088

In [39]:
correct = 0
wrong = []
for i in range(len(predictions)):
    if predictions[i] == test_labels[i]:
        correct += 1
    else:
        wrong.append([test_labels[i],predictions[i]])

In [40]:
percent_correct = round((correct/len(predictions))*100,2)

In [41]:
correct

53558

In [42]:
percent_correct

99.02

In [43]:
flagged_benign = 0
ignored_malicious = 0
for bad_prediction in wrong:
    if (bad_prediction[0] == 1) & (bad_prediction[1] == 0):
        flagged_benign += 1
    elif (bad_prediction[0] == 0) & (bad_prediction[1] == 1):
        ignored_malicious += 1
    else:
        print("ISSUE: ", bad_prediction)
        break

In [44]:
print(f"Accuracy of Classifier is: {percent_correct}%")
print(f"There were {len(wrong)} units wrongly classified out of {len(predictions)}")
print(f"We flagged {flagged_benign} benign files and let {ignored_malicious} malicious files slip by")

Accuracy of Classifier is: 99.02%
There were 530 units wrongly classified out of 54088
We flagged 286 benign files and let 244 malicious files slip by


## Feature Importance

In [23]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: ImageBase            Importance: 0.15
Variable: Characteristics      Importance: 0.1
Variable: SizeOfStackReserve   Importance: 0.08
Variable: MajorOperatingSystemVersion Importance: 0.06
Variable: Subsystem            Importance: 0.06
Variable: ResourcesMinSize     Importance: 0.06
Variable: ExportNb             Importance: 0.05
Variable: MajorSubsystemVersion Importance: 0.03
Variable: CheckSum             Importance: 0.03
Variable: SectionsMaxEntropy   Importance: 0.03
Variable: ResourcesMinEntropy  Importance: 0.03
Variable: MajorImageVersion    Importance: 0.02
Variable: MinorImageVersion    Importance: 0.02
Variable: ResourcesNb          Importance: 0.02
Variable: ResourcesMaxEntropy  Importance: 0.02
Variable: ResourcesMaxSize     Importance: 0.02
Variable: VersionInformationSize Importance: 0.02
Variable: MajorLinkerVersion   Importance: 0.01
Variable: MinorLinkerVersion   Importance: 0.01
Variable: SizeOfCode           Importance: 0.01
Variable: SizeOfInitializedData